In [1]:
import os
import json
import pandas as pd
import numpy as np
import tqdm
import scipy.sparse as sp

from pprint import pprint
from collections import defaultdict

DATA_PATH = 'D:\download'
with open(os.path.join(DATA_PATH, 'test_users.json'), 'r') as f:
    test_users = set(json.load(f)['users'])
    
transactions = pd.read_csv(
    os.path.join(DATA_PATH, 'transactions.csv'),
    dtype={
        'element_uid': np.uint16,
        'user_uid': np.uint32,
        'consumption_mode': 'category',
        'ts': np.float64,
        'watched_time': np.uint64,
        'device_type': np.uint8,
        'device_manufacturer': np.uint8
    })

ratings = pd.read_csv(
    os.path.join(DATA_PATH, 'ratings.csv'),
    dtype={
        'element_uid': np.uint16,
        'user_uid': np.uint32,
        'ts': np.float64,
        'rating': np.uint8
    }
)

    
ser = pd.read_csv("WTforserials(num3).csv")
mm = pd.read_csv("WTformnmov.csv")
OLD = pd.read_csv("WTforfilms(evr0204).csv")
rate=pd.read_csv('ratingsappend(notintransactions).csv')



OLD=OLD.append(ser)
OLD=OLD.append(rate)
OLD=OLD.append(mm)
#ratingbook=pd.read_csv("ratingbooksnew.csv")
#OLD=OLD.append(ratingbook)

In [4]:
ser = pd.read_csv("SeriesRF4NEWbezsubmitaworst.csv")
mm = pd.read_csv("MovRatingRFNEW3(doobuch)beznulltrain.csv")
OLD1 = pd.read_csv("MultMovRatingRF5(bezworst).csv")
rate=pd.read_csv('ratingsappend(notintransactions).csv')
OLD1=OLD1.append(ser)
OLD1=OLD1.append(rate)
OLD1=OLD1.append(mm)

In [5]:
OLD2=pd.merge(OLD, OLD1, how='left', on=['element_uid', 'user_uid'])
OLD2.head()

,Unnamed: 0_x,element_uid,rating_x,user_uid,Unnamed: 0_y,rating_y,worst
0,0,3336,9.0,5177,NaN,NaN,NaN
1,43,3336,8.0,86893,NaN,NaN,NaN
2,57,3336,10.0,419459,NaN,NaN,NaN
3,67,3336,7.0,184988,NaN,NaN,NaN
4,128,3336,8.0,521182,NaN,NaN,NaN


In [6]:
OLD2=OLD2.drop(['Unnamed: 0_x', 'Unnamed: 0_y', 'worst'],1)
OLD2.isnull().sum()/OLD2.shape[0]

element_uid    0.000000
rating_x       0.000000
user_uid       0.000000
rating_y       0.074043
dtype: float64

In [7]:
OLD2=OLD2.fillna(0)
OLD2['rating']=(OLD2['rating_x']+OLD2['rating_y'])/2
OLD2=OLD2.drop(['rating_x', 'rating_y'],1)
OLD2.head()

,element_uid,user_uid,rating
0,3336,5177,4.5
1,3336,86893,4.0
2,3336,419459,5.0
3,3336,184988,3.5
4,3336,521182,4.0


In [2]:
OLD2=OLD
OLD2=OLD2.drop(['Unnamed: 0'],1)
OLD2.isnull().sum()/OLD2.shape[0]

element_uid    0.0
rating         0.0
user_uid       0.0
dtype: float64

In [3]:
OLD2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7345862 entries, 0 to 118931
Data columns (total 3 columns):
element_uid    int64
rating         float64
user_uid       int64
dtypes: float64(1), int64(2)
memory usage: 224.2 MB


In [10]:
OLD2=OLD2.drop(['worst'],1)

In [4]:
SubsTran=pd.read_csv(os.path.join(DATA_PATH, 'SubsTran.csv'))
ratings=pd.merge(OLD2, SubsTran, how='left', on=['element_uid', 'user_uid'])
ratings.rating[~ratings.rate.isnull()]=ratings.rating[~ratings.rate.isnull()]+0.2

D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [5]:
ratings=ratings.drop(['Unnamed: 0', 'rate'], 1)
ratings.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7991557 entries, 0 to 7991556
Data columns (total 3 columns):
element_uid    int64
rating         float64
user_uid       int64
dtypes: float64(1), int64(2)
memory usage: 563.9 MB


In [7]:
OLD2=ratings

In [8]:

filtered_elements = defaultdict(set)
for user_uid, element_uid in tqdm.tqdm(transactions.loc[:, ['user_uid', 'element_uid']].values):
    if user_uid not in test_users:
        continue
    filtered_elements[user_uid].add(element_uid)
    

OLD2['user_uid'] = OLD2['user_uid'].astype('category')
OLD2['element_uid'] = OLD2['element_uid'].astype('category')
ratings_matrix = sp.coo_matrix(
    ((OLD2['rating'].astype(np.float32) + 1)**3,
        (
            OLD2['element_uid'].cat.codes.copy(),
            OLD2['user_uid'].cat.codes.copy()
        )
    )
)

ratings_matrix = ratings_matrix.tocsr()
sparsity = ratings_matrix.nnz / (ratings_matrix.shape[0] * ratings_matrix.shape[1])
print('Sparsity: %.6f' % sparsity)


100%|████████████████████████████| 9643012/9643012 [00:15<00:00, 621853.51it/s]


Sparsity: 0.001736


In [9]:
import implicit
from implicit.nearest_neighbours import TFIDFRecommender
model = TFIDFRecommender()
model.fit(ratings_matrix)
ratings_matrix_T = ratings_matrix.T.tocsr()
user_uid_to_cat = dict(zip(
    OLD2['user_uid'].cat.categories,
    range(len(OLD2['user_uid'].cat.categories))
))
element_uid_to_cat = dict(zip(
    OLD2['element_uid'].cat.categories,
    range(len(OLD2['element_uid'].cat.categories))
))
filtered_elements_cat = {k: [element_uid_to_cat.get(x, None) for x in v] for k, v in filtered_elements.items()}
result = {}

for user_uid in tqdm.tqdm(test_users):
    # transform user_uid to model's internal user category
    try:
        user_cat = user_uid_to_cat[user_uid]
    except LookupError:
        continue
    
    # perform inference
    recs = model.recommend(
        user_cat,
        ratings_matrix_T,
        N=20,
        filter_already_liked_items=True,
        filter_items=filtered_elements_cat.get(user_uid, set())
    )
    
    # drop scores and transform model's internal elelemnt category to element_uid for every prediction
    # also convert np.uint64 to int so it could be json serialized later
    result[user_uid] = [int(OLD2['element_uid'].cat.categories[i]) for i, _ in recs]
with open('answevr0304.json', 'w') as f:
    json.dump(result, f)

100%|██████████████████████████████████| 50000/50000 [00:13<00:00, 3616.95it/s]


In [10]:
with open(os.path.join( 'answevr0304.json'), 'r') as f:
    answ = (json.load(f))
with open(os.path.join( 'bookres(G1)4.0.json'), 'r') as f:
    book = (json.load(f))

    
bl=list(book.keys())
for key in bl:

    answ1=book.get('{}'.format(key))



    if len(answ1)==20:
        dct0={'{}'.format(key) : answ1}
        z=answ.update(dct0)
    else:
        answ2=answ.get('{}'.format(key))
        
        if answ2 == None:
            dct0={'{}'.format(key) : answ1}
            z=answ.update(dct0)
        else:
            x=20-len(answ1)
            answ3=answ2[:x]
            answ3=answ1+answ3
            
            if len(answ3) != len(list(dict.fromkeys(answ3))):
                t= len(answ3) - len(list(dict.fromkeys(answ3)))
                answ3=list(dict.fromkeys(answ3))
                answ3= answ2[x:(x+t)] + answ3
            dct0={'{}'.format(key) : answ3}
            z=answ.update(dct0)
with open('answ0304(3).json', 'w') as f:
    json.dump(answ, f)

In [22]:
book

{'121983': [6533, 4971, 8652, 6955, 8494, 4252, 4796],
 '418892': [6656, 8460, 9037, 5554, 9844, 4373, 2043],
 '42322': [2557, 153, 1653],
 '415948': [8330, 8420],
 '398628': [8739],
 '72789': [1090, 9540, 6471, 3722, 6126, 2357],
 '454547': [9057, 2949, 3045, 4972, 2447, 5488, 8530, 603],
 '172210': [6627, 7047, 6889, 2323, 6687],
 '23773': [8137],
 '545': [9025, 1227, 9932, 2510, 1653, 6585, 10171],
 '362534': [7522, 4525],
 '486731': [1650, 8436, 2293, 7190, 9341],
 '331095': [6213, 6726, 9836, 4493, 2702, 1519],
 '372875': [4591],
 '577486': [5921, 2278],
 '533187': [6436, 6213, 3877, 2138, 3376, 4915, 3188, 2650],
 '510765': [2694, 8647, 905, 4366, 2327, 5656, 1017, 4925],
 '398114': [4369, 1017, 2054],
 '216043': [5057, 2152, 361, 2923, 9939, 7381, 474],
 '461244': [6985, 1490],
 '474687': [10108],
 '385588': [7360, 6146, 3336, 8713, 6505, 4814, 9744, 849],
 '184247': [5592, 27, 2287],
 '428008': [3137, 9540, 8021, 294],
 '163981': [5616, 6214],
 '294763': [7360, 1570, 7913, 9077

In [21]:
count

660

In [2]:
with open(os.path.join( 'answevr0204.json'), 'r') as f:
    answ = (json.load(f))
with open(os.path.join( 'bookres(G1)4.0.json'), 'r') as f:
    book = (json.load(f))

    
bl=list(book.keys())
for key in bl:

    answ1=book.get('{}'.format(key))



    if len(answ1)==20:
        dct0={'{}'.format(key) : answ1}
        z=answ.update(dct0)
    else:
        answ2=answ.get('{}'.format(key))
        
        if answ2 == None:
            dct0={'{}'.format(key) : answ1}
            z=answ.update(dct0)
        else:
            x=20-len(answ1)
            
            if x%2==1:
                answ3=answ2[:x]
                answ3=answ3[:2]+answ1+answ3[2:x]
            else:
                answ3=answ2[:x]
                answ3=answ3[:2]+answ1+answ3[2:x]
            
            if len(answ3) != len(list(dict.fromkeys(answ3))):
                t= len(answ3) - len(list(dict.fromkeys(answ3)))
                answ3=list(dict.fromkeys(answ3))
                answ3=answ3 + answ2[x:(x+t)]
            dct0={'{}'.format(key) : answ3}
            z=answ.update(dct0)
with open('answ0304(2).json', 'w') as f:
    json.dump(answ, f)

In [25]:
with open(os.path.join( 'answ18.03_rf.json'), 'r') as f:
    answrf = (json.load(f))
with open(os.path.join( 'bookres(G1)4.0.json'), 'r') as f:
    book = (json.load(f))
with open(os.path.join( 'answ18.03.json'), 'r') as f:
    answxgb = (json.load(f))

    
    
    
bl=list(book.keys())
for key in bl:

    bookitems=book.get('{}'.format(key))



    if ((len(bookitems))%2) == 1:
        answ2=answrf.get('{}'.format(key))
        answ3=answxgb.get('{}'.format(key))
        if answ2 == None:
            x=20-len(bookitems)
            if answ3 == None:
                dct0={'{}'.format(key) : bookitems}
                z=answrf.update(dct0)
            else:
                answ3=answ3[:x]
                answ2=bookitems+answ3
                dct0={'{}'.format(key) : answ2}
                z=answrf.update(dct0)
        else:
            x=int(((len(bookitems))+1)/2)
            answ2=answ2[:x]
            answ3=answ3[:(x-1)]
            fin=bookitems+answ2+answ3
            dct0={'{}'.format(key) : fin}
            z=answrf.update(dct0)
        
        
    else:
        answ2=answrf.get('{}'.format(key))
        answ3=answxgb.get('{}'.format(key))        
        if answ2 == None:
            x=20-len(bookitems)
            if answ3 == None:
                dct0={'{}'.format(key) : bookitems}
                z=answrf.update(dct0)
            else:
                answ3=answ3[:x]
                answ2=bookitems+answ3
                dct0={'{}'.format(key) : answ2}
                z=answrf.update(dct0)
        else:
            x=int((len(bookitems))/2)
            if answ3 == None:
                x=len(bookitems)
                answ2=answ2[:x]
                fin=bookitems+answ2
                dct0={'{}'.format(key) : fin}
                z=answrf.update(dct0)
            else:
                answ2=answ2[:x]
                answ3=answ3[:x]
                fin=bookitems+answ2+answ3
                dct0={'{}'.format(key) : fin}
                z=answrf.update(dct0)    
  
with open('answ.18.03rfxgb.json', 'w') as f:
    json.dump(answrf, f)

In [26]:
answrf

{'0': [3230,
  9472,
  1636,
  10061,
  4141,
  3757,
  7079,
  9089,
  283,
  1004,
  427,
  905,
  1681,
  2030,
  1326,
  7449,
  3916,
  2714,
  5405,
  102],
 '131072': [6127,
  905,
  1016,
  7079,
  1004,
  5035,
  3916,
  1577,
  2360,
  8436,
  2714,
  3757,
  3783,
  4171,
  9491,
  5623,
  4290,
  1227,
  747,
  8647],
 '2': [2714,
  3336,
  4141,
  6195,
  1521,
  3916,
  2245,
  6209,
  3452,
  1016,
  6127,
  10061,
  71,
  3782,
  6377,
  813,
  153,
  549,
  9107,
  2639],
 '262145': [4441,
  3045,
  546,
  9179,
  6409,
  5554,
  6213,
  10061,
  3336,
  229,
  6127,
  6195,
  3856,
  2245,
  1521,
  1016,
  1570,
  813,
  9406,
  8771],
 '524291': [3567,
  5405,
  8215,
  5644,
  7233,
  9089,
  427,
  72,
  8501,
  3256,
  1681,
  57,
  9430,
  4519,
  2639,
  6377,
  1349,
  4096,
  102,
  6937],
 '131077': [72,
  3230,
  9472,
  5644,
  1636,
  8215,
  427,
  3947,
  51,
  540,
  3256,
  8501,
  102,
  5405,
  57,
  793,
  1349,
  7233,
  7636,
  3275],
 '6': [3782

In [27]:
with open(os.path.join( 'answ18.03_rf.json'), 'r') as f:
    answrf = (json.load(f))
answrf

{'0': [3230,
  9472,
  1636,
  10061,
  4141,
  3757,
  7079,
  9089,
  283,
  1004,
  427,
  905,
  1681,
  2030,
  1326,
  7449,
  3916,
  2714,
  5405,
  102],
 '131072': [6127,
  905,
  1016,
  7079,
  1004,
  5035,
  3916,
  1577,
  2360,
  8436,
  2714,
  3757,
  3783,
  4171,
  9491,
  5623,
  4290,
  1227,
  747,
  8647],
 '2': [2714,
  3336,
  4141,
  6195,
  1521,
  3916,
  2245,
  6209,
  3452,
  1016,
  6127,
  10061,
  71,
  3782,
  6377,
  813,
  153,
  549,
  9107,
  2639],
 '262145': [4441,
  3045,
  546,
  9179,
  6409,
  5554,
  6213,
  10061,
  3336,
  229,
  6127,
  6195,
  3856,
  2245,
  1521,
  1016,
  1570,
  813,
  9406,
  8771],
 '524291': [3567,
  5405,
  8215,
  5644,
  7233,
  9089,
  427,
  72,
  8501,
  3256,
  1681,
  57,
  9430,
  4519,
  2639,
  6377,
  1349,
  4096,
  102,
  6937],
 '131077': [72,
  3230,
  9472,
  5644,
  1636,
  8215,
  427,
  3947,
  51,
  540,
  3256,
  8501,
  102,
  5405,
  57,
  793,
  1349,
  7233,
  7636,
  3275],
 '6': [3782

In [28]:
answxgb

{'0': [9472,
  1636,
  3230,
  10061,
  4141,
  3757,
  9089,
  7079,
  283,
  1004,
  905,
  427,
  1681,
  2030,
  1326,
  7449,
  2714,
  5405,
  102,
  4519],
 '131072': [6127,
  905,
  7079,
  1016,
  1004,
  5035,
  1577,
  2714,
  3757,
  2360,
  4171,
  3916,
  3783,
  2639,
  5623,
  8436,
  9491,
  4290,
  8647,
  10084],
 '2': [2714,
  3336,
  4141,
  3452,
  6195,
  1521,
  3916,
  2245,
  6209,
  1016,
  6127,
  10061,
  71,
  6377,
  813,
  3782,
  153,
  549,
  2468,
  5616],
 '262145': [4441,
  3045,
  546,
  9179,
  6409,
  5554,
  6213,
  10061,
  3336,
  229,
  6195,
  6127,
  1521,
  2245,
  3856,
  1016,
  813,
  1570,
  8771,
  9406],
 '524291': [3567,
  5405,
  5644,
  8215,
  72,
  7233,
  9089,
  427,
  8501,
  3256,
  57,
  1681,
  4519,
  9430,
  2639,
  1349,
  6377,
  102,
  3250,
  3230],
 '131077': [72,
  3230,
  9472,
  5644,
  1636,
  8215,
  427,
  3947,
  51,
  540,
  3256,
  8501,
  102,
  5405,
  57,
  793,
  1349,
  7233,
  7636,
  2659],
 '6': [37

In [13]:
with open(os.path.join( 'answ29.03(for2mod).json'), 'r') as f:
    answrf = (json.load(f))

In [14]:
an=list(answrf.keys())
for key in an:

    answ1=answrf.get('{}'.format(key))
    answ2=answ1[:10]
    answ3=answ1[12:]
    answ4= answ2 + answ2
    dct0={'{}'.format(key) : answ4}
    z=answrf.update(dct0)

with open('answ.29(16).json', 'w') as f:
    json.dump(answrf, f)

In [12]:
answrf

{'0': [3757,
  10061,
  283,
  9472,
  1636,
  7449,
  1326,
  905,
  4141,
  427,
  9089,
  2694,
  1004,
  1650,
  5405,
  7400,
  8436,
  2714],
 '131072': [905,
  1577,
  1004,
  1016,
  3916,
  9491,
  1227,
  8436,
  2360,
  4171,
  2714,
  10084,
  4548,
  5623,
  2694,
  8647,
  3239,
  1521],
 '2': [71,
  6377,
  1636,
  3567,
  9089,
  2030,
  8501,
  9472,
  5405,
  5644,
  7233,
  4519,
  4978,
  5541,
  72,
  9430,
  1681,
  5798],
 '262145': [546,
  6409,
  6213,
  9179,
  10061,
  3336,
  5554,
  229,
  6195,
  1521,
  813,
  1570,
  8771,
  3856,
  6236,
  1538,
  2245,
  6127],
 '524291': [3256,
  5644,
  9089,
  7233,
  8215,
  8501,
  427,
  72,
  1681,
  9430,
  3329,
  4519,
  1349,
  57,
  6937,
  6662,
  3250,
  10032],
 '131077': [72,
  9472,
  3256,
  5644,
  3567,
  5405,
  1349,
  57,
  793,
  3230,
  51,
  289,
  3329,
  427,
  6173,
  7636,
  6961,
  4145],
 '6': [6127,
  1004,
  2760,
  3916,
  8195,
  1016,
  8898,
  905,
  7185,
  8981,
  3137,
  6955,
 

In [ ]:
remove(5)

In [ ]:
for user_uid in (test_users):
    if user_uid not in filtr:
        print(user_uid)

396764
275799
180884
328547
592316
202305
353063
247919
